In [1]:
import mygrad
import mygrad.functional as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = mygrad.value(X_train, "x_train", requires_grad=False)
X_test = mygrad.value(X_test, "x_test", requires_grad=False)
y_train = mygrad.value(y_train, "y_train", requires_grad=False)
y_test = mygrad.value(y_test, "y_test", requires_grad=False)

In [3]:
w = mygrad.value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = mygrad.value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F.matmul()
add1 = F.add()
loss = F.bce_loss_with_logits()
s = F.sigmoid()
    
def fit(num_epoch):     
    for i in range(num_epoch):        
        y = add1.forward(matmul1.forward(X_train, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.data -= lr * w.grad
        b.data -= lr * b.grad
        l.zero_grad()   
        
        print(f"Epoch {i}/{num_epoch}: Train Loss: {l.value}")

def predict(X, name):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = mygrad.value(s.forward(y).data > 0.5, name, requires_grad=False)
    
    return y_pred

Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_0
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_0
Creating Function of type <class 'mygrad.functional.function.BCELossWithLogits'> with name bce_loss_logit_0
Creating Function of type <class 'mygrad.functional.function.Sigmoid'> with name sigmoid_0


In [5]:
fit(100)    

Epoch 0/100: Train Loss: 0.6662056308383447
Epoch 1/100: Train Loss: 0.2812640712309442
Epoch 2/100: Train Loss: 0.14629777042084513
Epoch 3/100: Train Loss: 0.09267669323116429
Epoch 4/100: Train Loss: 0.07739961505153847
Epoch 5/100: Train Loss: 0.07113400749450911
Epoch 6/100: Train Loss: 0.0668627670912994
Epoch 7/100: Train Loss: 0.06372833176443544
Epoch 8/100: Train Loss: 0.06125972746123901
Epoch 9/100: Train Loss: 0.05920703800174774
Epoch 10/100: Train Loss: 0.057437619932931464
Epoch 11/100: Train Loss: 0.05587842951809782
Epoch 12/100: Train Loss: 0.0544862244274724
Epoch 13/100: Train Loss: 0.05323273905957752
Epoch 14/100: Train Loss: 0.052097561349876406
Epoch 15/100: Train Loss: 0.051064818340432284
Epoch 16/100: Train Loss: 0.050121641293221275
Epoch 17/100: Train Loss: 0.049257395709068466
Epoch 18/100: Train Loss: 0.04846321131430537
Epoch 19/100: Train Loss: 0.04773163020309174
Epoch 20/100: Train Loss: 0.047056318451967276
Epoch 21/100: Train Loss: 0.04643183035742

In [6]:
y_train_pred = predict(X_train, "y_train_pred")
y_test_pred = predict(X_test, "y_test_pred")

In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.value, y_train_pred.value)
print("")
print_metrics(y_test.value, y_test_pred.value)

Confusion Matrix
[[152   3]
 [  2 269]]

Accuracy: 0.9882629107981221
Precision: 0.9889705882352942
Recall: 0.992619926199262
AUC score: 0.9866325437447923
MCC score: 0.9746233835448521
F1-Score: 0.990791896869245


Confusion Matrix
[[54  3]
 [ 1 85]]

Accuracy: 0.972027972027972
Precision: 0.9659090909090909
Recall: 0.9883720930232558
AUC score: 0.9678702570379437
MCC score: 0.9417148216873074
F1-Score: 0.9770114942528736

